In [1]:
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm

In [2]:
country_data_path = '../Data/country_data'
macro_data_path = '../Data/macro_factors'

## Load Data

### Read country sovereign bond index data

In [3]:
ticker = pd.read_excel(os.path.join(country_data_path, 'ice_bofa_sovereign_indices_data.xlsx'), sheet_name='ticker mapping')

In [4]:
ticker['Country'] = ticker['Index Name'].apply(lambda x:x.split(' ')[4])

In [5]:
ticker['Index Full Name'] = ticker['Index'] + ' Index'

In [6]:
ticker[['Index Full Name', 'Country']].head()

,Index Full Name,Country
0,GDDZ Index,Algeria
1,GDAO Index,Angola
2,GDAR Index,Argentina
3,GDAM Index,Armenia
4,GDAW Index,Aruba


In [7]:
ticker_dict = pd.Series(data=ticker['Country'].values, index=ticker['Index Full Name']).to_dict()

In [8]:
bnd_idx_price = pd.read_excel(os.path.join(country_data_path, 'ice_bofa_sovereign_indices_data.xlsx'), 
                            sheet_name='px_last data',
                            skiprows=3,
                            header=0)

In [9]:
bnd_idx_price.drop(index=[0, 1], inplace=True)

In [10]:
bnd_idx_price.rename(columns={"Unnamed: 0":"Date"}, inplace=True)

In [11]:
bnd_idx_price.set_index('Date', inplace=True)

In [12]:
bnd_idx_price = bnd_idx_price.astype('float')

In [13]:
bnd_idx_price_monthly = bnd_idx_price.resample('M').last()

In [14]:
bnd_idx_price_monthly.replace(0, np.nan, inplace=True)

### Read Macro Factors Data

In [15]:
credit = pd.read_csv(os.path.join(macro_data_path, 'LF98TRUU_Index.csv'), header=None, parse_dates=[0], index_col=0, names=['credit'])
currency = pd.read_csv(os.path.join(macro_data_path, 'MXEF0CX0_Index.csv'), header=None, parse_dates=[0], index_col=0, names=['currency'])
equity = pd.read_csv(os.path.join(macro_data_path, 'WHANWIHD_Index.csv'), header=None, parse_dates=[0], index_col=0, names=['equity'])

In [16]:
rates = pd.read_csv(os.path.join(macro_data_path, 'JPM_GBIEM.csv'), skiprows=7, header=None, index_col=0, names=['rates_last', 'rates_mid'])
rates.index = pd.to_datetime(rates.index)
rates = rates.sort_index()
rates = rates[['rates_last']]
rates.columns = ['rates']

In [17]:
macro_factors = credit.join(currency, how='inner').join(equity, how='inner').join(rates, how='inner')

In [18]:
macro_factors_monthly = macro_factors.resample('M').last()

In [19]:
macro_factors_monthly = macro_factors_monthly.astype('float')

## Regression

In [20]:
bnd_index_ret = np.log(bnd_idx_price_monthly).diff()

In [21]:
bnd_index_ret.describe()

,GDAM Index,GDAO Index,GDAR Index,GDAW Index,GDAZ Index,GDBB Index,GDBG Index,GDBH Index,GDBO Index,GDBR Index,...,GDTH Index,GDTN Index,GDTR Index,GDTT Index,GDUA Index,GDUY Index,GDVE Index,GDVN Index,GDZA Index,GDZM Index
count,81.000000,101.000000,29.000000,99.000000,82.000000,168.000000,252.000000,211.000000,247.000000,252.000000,...,252.000000,225.000000,252.000000,193.000000,250.000000,252.000000,252.000000,193.000000,252.000000,99.000000
mean,0.006047,0.006631,-0.030350,0.004066,0.005400,0.003300,0.004364,0.000498,-0.028352,0.008063,...,0.002174,0.002953,0.007207,0.005278,0.009189,0.007198,0.005348,0.005765,0.006130,-0.000636
std,0.025372,0.105290,0.099993,0.015719,0.026430,0.034696,0.019377,0.004146,0.375968,0.042023,...,0.006958,0.012721,0.036664,0.022922,0.063254,0.057038,0.068971,0.030727,0.026784,0.080635
min,-0.135726,-0.893634,-0.338934,-0.069766,-0.132276,-0.165833,-0.147381,-0.028790,-1.386294,-0.210894,...,-0.026065,-0.072523,-0.180034,-0.184918,-0.479180,-0.381083,-0.552361,-0.281614,-0.191556,-0.457734
25%,-0.000509,-0.009947,-0.046797,0.000253,-0.006614,-0.004802,0.000000,0.000000,-0.223144,-0.008954,...,0.000000,0.000000,-0.009614,-0.000784,-0.008629,-0.008027,-0.013282,-0.001286,-0.006053,-0.031190
50%,0.007061,0.007067,-0.004017,0.004497,0.006777,0.000000,0.000000,0.000000,0.000000,0.011402,...,0.000000,0.000000,0.009581,0.006011,0.009573,0.008662,0.005048,0.007093,0.007877,-0.000797
75%,0.014938,0.024001,0.026629,0.007867,0.020004,0.012359,0.010756,0.000000,0.108293,0.025192,...,0.000511,0.006020,0.025770,0.014888,0.030000,0.031095,0.035912,0.017246,0.019398,0.028655
max,0.076157,0.337432,0.129239,0.071567,0.055869,0.170942,0.068749,0.022393,1.860752,0.233548,...,0.035111,0.062714,0.132061,0.084104,0.326299,0.297292,0.223687,0.099489,0.100748,0.327798


In [22]:
bnd_index_ret.replace(0, np.nan, inplace=True)

In [23]:
macro_factors_ret = np.log(macro_factors_monthly).diff().dropna()

In [24]:
macro_factors_ret.describe()

,credit,currency,equity,rates
count,191.000000,191.000000,191.000000,191.000000
mean,0.005827,0.002256,0.006560,0.003693
std,0.027960,0.019484,0.040996,0.012621
min,-0.173244,-0.078636,-0.171625,-0.039949
25%,-0.003483,-0.006882,-0.012876,-0.003229
50%,0.007352,0.003989,0.013603,0.005135
75%,0.016999,0.013584,0.030295,0.010941
max,0.114263,0.050544,0.114004,0.066541


In [25]:
results_df = pd.DataFrame()
results_dict = {} 

cols = ['const'] + macro_factors_ret.columns.tolist()
beta_cols = [x + '_beta' for x in cols]
tval_cols = [x + '_tval' for x in cols]

for bnd_index in bnd_index_ret:
    
    country = ticker_dict[bnd_index]
    temp = bnd_index_ret[bnd_index].dropna().to_frame().join(macro_factors_ret, how='inner')
    
    if temp.shape[0] > 30:

        y = temp.iloc[:, 0]
        X = temp.iloc[:, 1:]
        X = sm.add_constant(X)
        model = sm.OLS(y, X)
        results = model.fit()

        betas = results.params
        tvalues = results.tvalues
        rsq_adj = results.rsquared_adj
        f_pvalue = results.f_pvalue
        start_date = y.index.min().strftime('%Y%m')
        temp_results_df = pd.DataFrame(data=np.r_[betas, tvalues, rsq_adj, f_pvalue], 
                                       index=beta_cols + tval_cols + ['rsq_adj', 'f_pvalue']).T
        temp_results_df['country'] = country
        temp_results_df['start_date'] = start_date
        results_df = results_df.append(temp_results_df)
    
        results_dict[country] = results


In [26]:
results_df

,const_beta,credit_beta,currency_beta,equity_beta,rates_beta,const_tval,credit_tval,currency_tval,equity_tval,rates_tval,rsq_adj,f_pvalue,country,start_date
0,0.000305,0.686393,-0.032548,0.049879,0.685186,0.167823,4.709735,-0.239380,0.685130,3.230734,0.657174,2.323786e-17,Armenia,201405
0,-0.013690,4.675250,-0.339382,-0.280365,-0.331865,-1.884888,7.549322,-0.576483,-0.901290,-0.401961,0.584563,3.154663e-18,Angola,201209
0,0.002678,0.213064,0.186602,0.019919,0.017648,1.730829,1.629997,1.485496,0.302846,0.101022,0.173711,2.108837e-04,Aruba,201211
0,0.001345,0.867756,0.291463,-0.061860,0.219105,0.711122,5.664821,2.034744,-0.806549,0.979627,0.643828,3.419223e-17,Azerbaijan,201404
0,0.000546,0.020166,-0.027514,0.139675,0.991165,0.159068,0.114191,-0.115472,1.076067,3.385233,0.148544,9.684016e-05,Barbados,200702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,-0.002072,0.684893,-0.157010,-0.068228,1.811656,-0.951549,5.610834,-1.009153,-0.835528,8.909230,0.557713,8.343275e-33,Uruguay,200502
0,-0.003530,0.693172,1.082053,0.074153,-0.234718,-0.564180,2.004747,2.382286,0.295569,-0.411860,0.182031,4.002380e-07,Venezuela,200502
0,-0.000700,0.570394,-0.075296,-0.043229,0.944979,-0.417491,6.073829,-0.629051,-0.688111,6.040457,0.490769,3.644347e-27,Vietnam,200502
0,-0.001862,0.580584,-0.131527,-0.083354,1.188352,-1.403447,7.807907,-1.387746,-1.675678,9.593434,0.632474,3.122304e-40,South,200502


In [28]:
results_df.describe()

,const_beta,credit_beta,currency_beta,equity_beta,rates_beta,const_tval,credit_tval,currency_tval,equity_tval,rates_tval,rsq_adj,f_pvalue
count,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,6.400000e+01
mean,-0.001206,0.743044,0.114240,-0.072579,0.571318,0.334866,4.579336,0.095035,-0.805315,4.051244,0.423691,1.477038e-02
std,0.007998,0.807049,0.567000,0.190828,0.552946,1.376427,2.752452,1.410407,1.143537,3.475218,0.192082,8.040200e-02
min,-0.055566,-0.271067,-0.415031,-1.351163,-1.582997,-2.327720,-0.994057,-3.410316,-3.788436,-0.798679,-0.029293,1.282231e-47
25%,-0.001915,0.264743,-0.115477,-0.097988,0.266455,-0.472015,2.144931,-0.981624,-1.564954,0.976809,0.269759,2.269891e-28
50%,0.000606,0.528850,-0.014437,-0.055227,0.576591,0.304527,5.253738,-0.080361,-0.728029,3.510033,0.466293,6.359690e-18
75%,0.001724,0.942705,0.186721,-0.008124,0.922791,1.264359,6.454711,0.951356,-0.102838,6.432483,0.572426,9.649628e-10
max,0.010363,4.675250,3.990424,0.184281,1.811656,3.328295,10.136082,3.721224,1.906052,11.945734,0.694192,6.193399e-01
